### A symbolic Jacobian for a stiff ODE system

This example is based on an old scipy example script for a chemical reaction system of three concentrations. This is a very stiff system as it involves very disparate time scales (see the magnitudes of the coefficients below). We solve this using the 'stiff' option of VODE, taking advantage of an explicit Jacobian matrix function:

$$\frac{dy_o}{dt}=-0.04 y_o + 10^{-4} y_1 y_2 $$
$$ \frac{dy_1}{dt} = 3 \cdot 10^7 y_1^2 $$
$$ \frac{dy_2}{dt} = -\frac{dy_o}{dt}-\frac{dy_1}{dt} $$

In [1]:
from PyDSTool import *

Declarations of symbolic objects

In [2]:
y0 = Var('y0')
y1 = Var('y1')
y2 = Var('y2')
t = Var('t')

Definitions of right-hand sides for each variable, as functions of all variables

In [3]:
ydot0 = Fun(-0.04*y0 + 1e4*y1*y2, [y0, y1, y2], 'ydot0')
ydot2 = Fun(3e7*y1*y1, [y0, y1, y2], 'ydot2')
ydot1 = Fun(-ydot0(y0,y1,y2)-ydot2(y0,y1,y2), [y0, y1, y2], 'ydot1')

The whole vector field as one nonlinear multi-variate function: $R^3 -> R^3$

In [4]:
F = Fun([ydot0(y0,y1,y2),ydot1(y0,y1,y2),ydot2(y0,y1,y2)], [y0,y1,y2], 'F')


Diff is the symbolic derivative operator, and returns a QuantSpec object.
It must be named "Jacobian" for PyDSTool to recognize it as such.

In [5]:
jac = Fun(Diff(F, [y0,y1,y2]), [t, y0, y1, y2], 'Jacobian')

This defines a function of arguments (t, y0, y1, y2) obtained by differentiating F in all three variables, y0, y1, and y2.

The user could inspect this expression, jac, and make some simplifications
by hand, to help optimize the speed of its evaluation. Here, we'll
just use it as is.

In [6]:
DSargs = args(name='jactest', checklevel=2)
DSargs.fnspecs = [jac, ydot0, ydot2]
DSargs.varspecs = {y0: ydot0(y0,y1,y2),
                   y2: ydot2(y0,y1,y2),
                   y1: -ydot0(y0,y1,y2)-ydot2(y0,y1,y2)}
DSargs.tdomain = [0.,1e20]
DSargs.ics = {y0: 1.0, y1: 0., y2: 0.}
DSargs.algparams = {'init_step':0.4, 'strictdt': True, 'stiff': True,
                    'rtol': 1e-4, 'atol': [1e-8,1e-14,1e-6]}
DSargs.events = makeZeroCrossEvent(y0-0.001, -1, {'name': 'thresh_ev',
                       'eventtol': 10,
                       'bisectlimit': 20,
                       'eventinterval': 500,
                       'eventdelay': 0,  #otherwise cannot catch event with only one step per run
                       'term': True}, [y0])

In [7]:
testODE = Vode_ODEsystem(DSargs)

In [8]:
print ("Defined the following internal Python function for Jacobian:")
print (testODE.funcspec.auxfns['Jacobian'][0])

print ("\nOutput at exponentially larger subsequent time steps (with event detection!):\n")

Defined the following internal Python function for Jacobian:
def _auxfn_Jac(ds, t, x, parsinps):
    xjac0 = [-0.04,10000*x[2],10000*(x[1])] 
    xjac1 = [0.04,-10000*(x[2])-30000000*(2*x[1]),-10000*(x[1])] 
    xjac2 = [0,30000000*(2*x[1]),0] 
    return array([xjac0, xjac1, xjac2])

Output at exponentially larger subsequent time steps (with event detection!):



In [9]:
tvals = [0.4*10**i for i in range(0,12)]
t0 = 0.

for t1 in tvals:
    dt = t1-t0
    print ("\n===============================================\nAt t=",
           t1, "using dt =", dt)
    testODE.set(tdata=[t0,t1], algparams={'init_step': dt})
    if t0 >0.:
        print (testODE._solver.y)
    traj = testODE.compute('test', 'c')  # c for continue (acts as f first time)
    try:
        print (traj(t1))
    except ValueError:
        print ("\n----------")
        testODE.diagnostics.showWarnings()
        print (traj(traj.indepdomain[1]))
        print ("----------\n")
        t0 = traj.indepdomain[1]
    else:
        t0 = t1


At t= 0.4 using dt = 0.4
y0:  0.985169715618
y1:  3.38634977697e-05
y2:  0.0147964208839

At t= 4.0 using dt = 3.6
[  9.85169716e-01   3.38634978e-05   1.47964209e-02]
y0:  0.905516381021
y1:  2.24042194094e-05
y2:  0.0944612147593

At t= 40.0 using dt = 36.0
[  9.05516381e-01   2.24042194e-05   9.44612148e-02]
y0:  0.715843348571
y1:  9.18595678038e-06
y2:  0.284147465472

At t= 400.0 using dt = 360.0
[  7.15843349e-01   9.18595678e-06   2.84147465e-01]
y0:  0.450536684597
y1:  3.22309525784e-06
y2:  0.549460092308

At t= 4000.0 using dt = 3600.0
[  4.50536685e-01   3.22309526e-06   5.49460092e-01]
y0:  0.183212457766
y1:  8.94108277768e-07
y2:  0.816786648126

At t= 40000.0 using dt = 36000.0
[  1.83212458e-01   8.94108278e-07   8.16786648e-01]
y0:  0.0389906130057
y1:  1.62208803942e-07
y2:  0.961009224786

At t= 400000.0 using dt = 360000.0
[  3.89906130e-02   1.62208804e-07   9.61009225e-01]
y0:  0.00493843273719
y1:  1.9850503499e-08
y2:  0.995061547412

At t= 4000000.0 using dt

In [10]:
print ("\nCompare results with the output directly from the scipy_ode.py test")
print ("(up to the point where the terminal event was found).")
print ("The values from a test integration performed with scipy_ode.py " \
      + "are listed in the comments at the end of this script")

## At t=0.0  y=[ 1.  0.  0.]
## At t=0.4  y=[ 9.85172114e-001  3.38639538e-005  1.47940221e-002]
## At t=4.0  y=[ 9.05518679e-001  2.24047569e-005  9.44589166e-002]
## At t=40.0  y=[ 7.15827070e-001  9.18553479e-006  2.84163745e-001]
## At t=400.0  y=[ 4.50518662e-001  3.22290136e-006  5.49478115e-001]
## At t=4000.0  y=[ 1.83202242e-001  8.94237031e-007  8.16796863e-001]
## At t=40000.0  y=[ 3.89833646e-002  1.62176779e-007  9.61016473e-001]
## At t=400000.0  y=[ 4.93828363e-003  1.98499788e-008  9.95061697e-001]
## At t=4000000.0  y=[ 5.16819063e-004  2.06833253e-009  9.99483179e-001]
## At t=40000000.0  y=[ 5.20200703e-005  2.08090952e-010  9.99947980e-001]
## At t=400000000.0  y=[ 5.21215549e-006  2.08487297e-011  9.99994788e-001]
## At t=4000000000.0  y=[ 5.25335126e-007  2.10134087e-012  9.99999475e-001]
## At t=40000000000.0  y=[ 5.63729748e-008  2.25491907e-013  9.99999944e-001]


Compare results with the output directly from the scipy_ode.py test
(up to the point where the terminal event was found).
The values from a test integration performed with scipy_ode.py are listed in the comments at the end of this script
